# Getting started

In this notebook, we illustrate how to use the Neural News Recommendation with Multi-Head Self-Attention ([NRMS](https://aclanthology.org/D19-1671/)). The implementation is taken from the [recommenders](https://github.com/recommenders-team/recommenders) repository. We have simply stripped the model to keep it cleaner.

We use a small dataset, which is downloaded from [recsys.eb.dk](https://recsys.eb.dk/). All the datasets are stored in the folder path ```~/ebnerd_data/*```.

## Load functionality

In [1]:
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import tensorflow as tf
import polars as pl
import shutil
import os
import itertools

from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_LABELS_COL,
    DEFAULT_TITLE_COL,
    DEFAULT_USER_COL,
)

from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    add_known_user_column,
    add_prediction_scores,
    truncate_history,
)
from ebrec.utils._articles import convert_text2encoding_with_transformers
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._articles import create_article_id_to_value_mapping
from ebrec.utils._nlp import get_transformers_word_embeddings
from ebrec.utils._python import write_submission_file, rank_predictions_by_score

from ebrec.models.newsrec.dataloader import LSTURDataLoader
from ebrec.models.newsrec.model_config import hparams_lstur
from ebrec.models.newsrec import LSTURModel


2024-06-19 18:38:29.079721: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 18:38:29.079814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 18:38:29.083315: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-19 18:38:29.102984: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 18:38:30.607070: W tensorflow/comp

## Load dataset

In [2]:
def ebnerd_from_path(path: Path, history_size: int = 30) -> pl.DataFrame:
    """
    Load ebnerd - function
    """
    df_history = (
        pl.scan_parquet(path.joinpath("history.parquet"))
        .select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL)
        .pipe(
            truncate_history,
            column=DEFAULT_HISTORY_ARTICLE_ID_COL,
            history_size=history_size,
            padding_value=0,
            enable_warning=False,
        )
    )
    df_behaviors = (
        pl.scan_parquet(path.joinpath("behaviors.parquet"))
        .collect()
        .pipe(
            slice_join_dataframes,
            df2=df_history.collect(),
            on=DEFAULT_USER_COL,
            how="left",
        )
    )
    return df_behaviors

### Generate labels
We sample a few just to get started. For testset we just make up a dummy column with 0 and 1 - this is not the true labels.

In [3]:
PATH = Path("../data/")
DATASPLIT = "ebnerd_small"

In this example we sample the dataset, just to keep it smaller. Also, one can simply add the testset similary to the validation.

In [4]:
COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
]
HISTORY_SIZE = 10
FRACTION = 0.01

df_train = (
    ebnerd_from_path(PATH.joinpath(DATASPLIT, "train"), history_size=HISTORY_SIZE)
    .select(COLUMNS)
    .pipe(
        sampling_strategy_wu2019,
        npratio=4,
        shuffle=True,
        with_replacement=True,
        seed=123,
    )
    .pipe(create_binary_labels_column)
    .sample(fraction=FRACTION)
)
# =>
df_validation = (
    ebnerd_from_path(PATH.joinpath(DATASPLIT, "validation"), history_size=HISTORY_SIZE)
    .select(COLUMNS)
    .pipe(create_binary_labels_column)
    .sample(fraction=FRACTION)
)
df_train.head(2)

user_id,article_id_fixed,article_ids_inview,article_ids_clicked,impression_id,labels
u32,list[i32],list[i64],list[i64],u32,list[i8]
2433383,"[9762288, 9770400, … 9770327]","[7420751, 9774187, … 7420751]",[9774187],451574915,"[0, 1, … 0]"
1332809,"[9768819, 9768829, … 9770146]","[9770082, 9771796, … 9771796]",[9770082],375736276,"[1, 0, … 0]"


## Load articles

In [5]:
df_articles = pl.read_parquet(PATH.joinpath(DATASPLIT, "articles.parquet"))
df_articles.head(2)

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var i…","""Politiet frygt…",2023-06-29 06:20:33,false,"""Sagen om den ø…",2006-08-31 08:06:45,[3150850],"""article_defaul…","""https://ekstra…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars …","""Biografgængern…",2023-06-29 06:20:35,false,"""Vatikanet har …",2006-05-21 16:57:00,[3006712],"""article_defaul…","""https://ekstra…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""


## Init model using HuggingFace's tokenizer and wordembedding
In the original implementation, they use the GloVe embeddings and tokenizer. To get going fast, we'll use a multilingual LLM from Hugging Face. 
Utilizing the tokenizer to tokenize the articles and the word-embedding to init NRMS.


In [6]:
TRANSFORMER_MODEL_NAME = "FacebookAI/xlm-roberta-base"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 50

# LOAD HUGGINGFACE:
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# We'll init the word embeddings using the
word2vec_embedding = get_transformers_word_embeddings(transformer_model)
#
df_articles, cat_cal = concat_str_columns(df_articles, columns=TEXT_COLUMNS_TO_USE)
df_articles, token_col_title = convert_text2encoding_with_transformers(
    df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
# =>
article_mapping = create_article_id_to_value_mapping(
    df=df_articles, value_col=token_col_title
)

/home/jtaha/recsys/group-33-8Iui3YMO-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Initiate the dataloaders
In the implementations we have disconnected the models and data. Hence, you should built a dataloader that fits your needs.

In [7]:
user_id_mapping = {user_id: i for i, user_id in enumerate(df_train[DEFAULT_USER_COL].unique())}

train_dataloader = LSTURDataLoader(
    user_id_mapping=user_id_mapping,
    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=64,
)
val_dataloader = LSTURDataLoader(
    user_id_mapping=user_id_mapping,
    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=32,
)

# Performing Hyperopt
First we perform hyperopt in input dimensions namely: hoistory_size, user_size and title_size

In [8]:
def objective(history_size, n_users, title_size, learning_rate, dropout, df_validation,df_train):

    # Create directories for logs and model weights
    MODEL_NAME = f"LSTUR_l{learning_rate}_d{dropout}"
    LOG_DIR = f"downloads/runs/{MODEL_NAME}"
    MODEL_WEIGHTS = f"downloads/data/state_dict/{MODEL_NAME}/weights"
    RESULTS_DIR = f"downloads/evaluations/{MODEL_NAME}"
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
    modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=MODEL_WEIGHTS, save_best_only=True, save_weights_only=True, verbose=1
    )

    # Set the parameters
    hparams_lstur.history_size = history_size
    hparams_lstur.n_users = n_users
    hparams_lstur.title_size = title_size
    hparams_lstur.learning_rate = learning_rate
    hparams_lstur.dropout = dropout
    

    model = LSTURModel(
        hparams=hparams_lstur,
        word2vec_embedding=word2vec_embedding,
        seed=42,
    )
    hist = model.model.fit(
        train_dataloader,
        validation_data=val_dataloader,
        epochs=1,
        callbacks=[tensorboard_callback, early_stopping, modelcheckpoint],
    )
    _ = model.model.load_weights(filepath=MODEL_WEIGHTS)


    
    pred_validation = model.scorer.predict(val_dataloader)
    df_validation = add_prediction_scores(df_validation, pred_validation.tolist()).pipe(
        add_known_user_column, known_users=df_train[DEFAULT_USER_COL]
    )

    from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
    
    metrics = MetricEvaluator(
        labels=df_validation["labels"].to_list(),
        predictions=df_validation["scores"].to_list(),
        metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
    )
    evaluation_results = metrics.evaluate().evaluations

    # Save the evaluation results
    os.makedirs(RESULTS_DIR, exist_ok=True)
    with open(os.path.join(RESULTS_DIR, 'evaluation_results.txt'), 'w') as f:
        for key, value in evaluation_results.items():
            f.write(f"{key}: {value}\n")

    # Rank predictions and write submission file
    df_validation = df_validation.with_columns(
        pl.col("scores")
        .map_elements(lambda x: list(rank_predictions_by_score(x)))
        .alias("ranked_scores")
    )
    write_submission_file(
        impression_ids=df_validation[DEFAULT_IMPRESSION_ID_COL],
        prediction_scores=df_validation["ranked_scores"],
        path=os.path.join(RESULTS_DIR, "predictions.txt"),
    )

    return evaluation_results

In [9]:
param_grid = {
    'history_size': [100],
    'n_users':  [70000],
    'title_size': [50],
    'learning_rate': [0.0001, 0.001, 0.01],
    'dropout':  [0.1, 0.3, 0.5]
}

combinations = list(
    itertools.product(
        param_grid['history_size'], param_grid['n_users'], param_grid['title_size'], param_grid['learning_rate'], param_grid['dropout']
    )
)


all_results = []
for history_size, n_users, title_size, learning_rate, dropout in combinations:
    print(f"Evaluating combination: history_size={history_size}, n_users={n_users}, title_size={title_size}")
    result = objective(history_size, n_users, title_size, learning_rate, dropout, df_validation, df_train)
    all_results.append({
        'history_size': history_size,
        'n_users': n_users,
        'title_size': title_size,
        'evaluation_results': result
    })

# Save all results to a file
with open("downloads/evaluations/all_results.txt", 'w') as f:
    for result in all_results:
        f.write(f"{result}\n")

print("All combinations evaluated.")


Evaluating combination: history_size=100, n_users=70000, title_size=50


2024-06-19 18:39:03.339396: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-19 18:39:03.345001: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-19 18:39:03.348078: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

KerasTensor(type_spec=TensorSpec(shape=(None, 400), dtype=tf.float32, name=None), name='att_layer2/Sum_1:0', description="created by layer 'att_layer2'")


2024-06-19 18:39:07.941617: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-19 18:39:15.400662: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-06-19 18:39:15.670208: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-19 18:39:18.300474: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7bd5fac8f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-19 18:39:18.300548: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-06-19 18:39:18.316275: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1718822358.523325   11580 device_compiler.h:186] Compiled

37/37 [==============================] - ETA: 0s - loss: 1.5978

...

Zipping downloads/evaluations/LSTUR_l0.01_d0.1/predictions.txt to downloads/evaluations/LSTUR_l0.01_d0.1/predictions.zip
Evaluating combination: history_size=100, n_users=70000, title_size=50
KerasTensor(type_spec=TensorSpec(shape=(None, 400), dtype=tf.float32, name=None), name='att_layer2_7/Sum_1:0', description="created by layer 'att_layer2_7'")
37/37 [==============================] - ETA: 0s - loss: 3.4180
Epoch 1: val_loss improved from inf to 0.00000, saving model to downloads/data/state_dict/LSTUR_l0.01_d0.3/weights
77/77 [==============================] - 14s 158ms/step


2446it [00:00, 11429.53it/s]


Zipping downloads/evaluations/LSTUR_l0.01_d0.3/predictions.txt to downloads/evaluations/LSTUR_l0.01_d0.3/predictions.zip
Evaluating combination: history_size=100, n_users=70000, title_size=50
KerasTensor(type_spec=TensorSpec(shape=(None, 400), dtype=tf.float32, name=None), name='att_layer2_8/Sum_1:0', description="created by layer 'att_layer2_8'")
37/37 [==============================] - ETA: 0s - loss: 2.9164
Epoch 1: val_loss improved from inf to 0.00000, saving model to downloads/data/state_dict/LSTUR_l0.01_d0.5/weights
77/77 [==============================] - 15s 156ms/step


2446it [00:00, 11181.53it/s]

Zipping downloads/evaluations/LSTUR_l0.01_d0.5/predictions.txt to downloads/evaluations/LSTUR_l0.01_d0.5/predictions.zip
All combinations evaluated.


# DONE 🚀